<font size=6 color='blue'>

<center> Clase 5, octubre 20 del 2021</center>

<font size=4 color='blue'>
    
## Problema a resolver


<img src="./images/Problema.jpg" width=420 height=420 align = "center" >

<font size=6 color='blue'>
Mortalidad por diabetes

<img src="./images/Diabetes.png" width=420 height=420 align = "center" >

<font size=5 color='blue'>
Información sobre el problema a resolver

<font size=4>

Evolución de la enfermedad de pacientes con Diabetes Mellitus despues de un año.
    
En el presente trabajo, la diabetes la caracterizamos con los siguientes diez rasgos: edad, sexo, índice de masa corporal, presión arterial promedio y seis mediciones de suero sanguíneo:

     Colesterol Total 
     Baja densidad de liporoteinas
     Alta densidad de lipoproteinas
     Triglicéridos
     Concentración de Lamorigina
     Glucosa

<font size=4 color='blue'>
    
## Cuantificación de esta información

<font size=4>

Se tienen información de 442 pacientes (m = 442). La respuesta de interés, Y, es una medida cuantitativa de la progresión de la enfermedad un año después del inicio del estudio. Los valores de Y varían entre 25 y 346

Fuente de la información: [diabetes data](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html)    Artículo original: [Least-Angle-Regression_2004](./Literatura/Least-Angle-Regression_2004.pdf)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

np.random.seed(1)

In [ ]:
# Los datos se encuentran el el archivo diabetes.csv

df = pd.read_csv('diabetes.csv', sep ='\t')

# se crea el dataframe df, el cual contiene los 10 rasgos relevantes de los pacientes
# diabeticos, así como el progreso (y) de la enfermedad un año después de comenzado el estudio. 

In [ ]:
# Se despliegan las primeras 5 muestras (rasgos, objetivo)

df.head()

<font size=4>
Las abreviaciones tienen el siguiente significado:
    
    AGE = Age
    SEX = Sex
    BMI = Body Mass Index (BMI)
     BP = Mean Arterial Pressure (MAP)
     S1 = Total Cholesterol (TC)
     S2 = Low Density lipoproteins (LDL)
     S3 = High Density lipoproteins (HDL)
     S4 = Triglyceride (TG, TCH)
     S5 = Serum Concentration of Lamorigine (LTG)
     S6 = Glucose (GLU)
     Y = Quantitative Measure of Diabetes Mellitus Disease Progression (QMDMDP) one year after the baseline.

In [ ]:
# el método describe() genera una tabla con informacion estadistica de cada uno de los rasgos y del objetivo.

df.describe()

## Se crean los histogramas para cada uno de los rasgos que caracteriza a los pacientes con diabetes:

In [ ]:
plt.figure(figsize=(20,8)) 

ax1 = plt.subplot(2,4,1)
ax2 = plt.subplot(2,4,2)
ax3 = plt.subplot(2,4,3)
ax4 = plt.subplot(2,4,4)

ax1.hist(df.AGE, bins=30, color='green',edgecolor='purple', alpha=0.5)
ax1.set_xlabel('Age (years)', size=15)
ax1.set_ylabel('Frequency', size=15)

ax2.hist(df.SEX, bins=30, color='orange',edgecolor='purple', alpha=0.5)
ax2.set_xlabel('Sex', size=15)

ax3.hist(df.BMI, bins=30, color='red',edgecolor='purple', alpha=0.5)
ax3.set_xlabel('Body_mass_index', size=15)

ax4.hist(df.BP, bins=30, color='blue',edgecolor='purple', alpha=0.5)
ax4.set_xlabel('Mean_Arterial_Pressure', size=15);

In [ ]:
plt.figure(figsize=(20,8)) 

ax1 = plt.subplot(2,4,1)
ax2 = plt.subplot(2,4,2)
ax3 = plt.subplot(2,4,3)
ax4 = plt.subplot(2,4,4)

ax1.hist(df.S1, bins=30, color='green',edgecolor='purple', alpha=0.5)
ax1.set_xlabel('Total Cholesterol', size=15)
ax1.set_ylabel('Frequency', size=15)

ax2.hist(df.S2, bins=30, color='orange',edgecolor='purple', alpha=0.5)
ax2.set_xlabel('Low Density lipoproteins', size=15)

ax3.hist(df.S3, bins=30, color='red',edgecolor='purple', alpha=0.5)
ax3.set_xlabel('High Density lipoproteins', size=15)

ax4.hist(df.S4, bins=30, color='blue',edgecolor='purple', alpha=0.5)
ax4.set_xlabel('Triglyceride', size=15);

In [ ]:
plt.figure(figsize=(15,8)) 

ax1 = plt.subplot(2,3,1)
ax2 = plt.subplot(2,3,2)
ax3 = plt.subplot(2,3,3)

ax1.hist(df.S5, bins=30, color='green',edgecolor='purple', alpha=0.5)
ax1.set_xlabel('Serum Concentration of Lamorigine', size=15)
ax1.set_ylabel('Frequency', size=15)

ax2.hist(df.S6, bins=30, color='orange',edgecolor='purple', alpha=0.5)
ax2.set_xlabel('Glucose', size=15)

ax3.hist(df.Y, bins=30, color='purple',edgecolor='black', alpha=0.5)
ax3.set_xlabel('Y(Diabetes Mellitus Disease Progression)', size=15)


<font size=4>

Para quitar cualquier posible correlación entre las muestras (los renglones del DataFrame), estos se reordenan al azar.

In [ ]:
df = df.sample(frac=1)

<font size=5 color='blue'>
División de las muestras para aprender y para hacer predicciones

<font size=4>
    
Se dividen la muestras originales en 2 conjuntos: 90 % para el entrenamiento y 10 % para hacer inferencias (predicciones) con el sistema de aprendizaje.

In [ ]:
test_ratio = 0.1

train_ratio = int((1.0-test_ratio)*len(df.values[:,:]))

df_train = df.iloc[0:train_ratio,:]
df_test  = df.iloc[train_ratio:,:]

In [ ]:
print(df_train.shape)
print(df_test.shape)

<font size=4>

Para trabajar con los modelos de aprendizaje,es adecuado que todas las variables tengan el mismo orden de magnitud. Por ello, se normalizan sus valores en las muestras que se emplearán en el entrenamiento, tanto los rasgos (X) y las variables objetivo (Y):

$$x_{i,norm} = \dfrac{x_{i}-\mu}{\sigma}$$
    
$$y_{i,norm} = \dfrac{y_{i}-\mu}{\sigma}$$

In [ ]:
df_train_norm = (df_train - df_train.mean()) / df_train.std()

In [ ]:
df_test_norm = (df_test - df_train.mean()) / df_train.std()

<font size=4>
    
Histogramas de las variables que se emplearán en el entrenamiento:

In [ ]:
plt.figure(figsize=(20,8)) 

ax1 = plt.subplot(2,4,1)
ax2 = plt.subplot(2,4,2)
ax3 = plt.subplot(2,4,3)
ax4 = plt.subplot(2,4,4)

ax1.hist(df_train_norm.AGE, bins=30, color='green',edgecolor='purple', alpha=0.5)
ax1.set_xlabel('x1(Age)', size=15)
ax1.set_ylabel('Frequency', size=15)

ax2.hist(df_train_norm.SEX, bins=30, color='orange',edgecolor='purple', alpha=0.5)
ax2.set_xlabel('x2(Sex)', size=15)

ax3.hist(df_train_norm.BMI, bins=30, color='red',edgecolor='purple', alpha=0.5)
ax3.set_xlabel('x3(Body_mass_index)', size=15)

ax4.hist(df_train_norm.BP, bins=30, color='blue',edgecolor='purple', alpha=0.5)
ax4.set_xlabel('x4(Mean_Arterial_Pressure)', size=15);

In [ ]:
plt.figure(figsize=(20,8)) 

ax1 = plt.subplot(2,4,1)
ax2 = plt.subplot(2,4,2)
ax3 = plt.subplot(2,4,3)
ax4 = plt.subplot(2,4,4)

ax1.hist(df_train_norm.S1, bins=30, color='green',edgecolor='purple', alpha=0.5)
ax1.set_xlabel('x5(Total Cholesterol)', size=15)
ax1.set_ylabel('Frequency', size=15)

ax2.hist(df_train_norm.S2, bins=30, color='orange',edgecolor='purple', alpha=0.5)
ax2.set_xlabel('x6(Low Density lipoproteins)', size=15)

ax3.hist(df_train_norm.S3, bins=30, color='red',edgecolor='purple', alpha=0.5)
ax3.set_xlabel('x7(High Density lipoproteins)', size=15)

ax4.hist(df_train_norm.S4, bins=30, color='blue',edgecolor='purple', alpha=0.5)
ax4.set_xlabel('x8(Triglyceride)', size=15);

In [ ]:
plt.figure(figsize=(20,8)) 

ax1 = plt.subplot(2,3,1)
ax2 = plt.subplot(2,3,2)
ax3 = plt.subplot(2,3,3)

ax1.hist(df_train_norm.S5, bins=30, color='green',edgecolor='purple', alpha=0.5)
ax1.set_xlabel('x9(Serum Concentration of Lamorigine)', size=15)
ax1.set_ylabel('Frequency', size=15)

ax2.hist(df_train_norm.S6, bins=30, color='orange',edgecolor='purple', alpha=0.5)
ax2.set_xlabel('x10(Glucose)', size=15)

ax3.hist(df_train_norm.Y, bins=30, color='purple',edgecolor='black', alpha=0.5)
ax3.set_xlabel('Y(Diabetes Mellitus Disease Progression)', size=15)


In [ ]:
train_x = df_train_norm.values[:,:-1]
train_y = df_train_norm.values[:,-1:]

In [ ]:
test_x = df_test_norm.values[:,:-1]
test_y = df_test_norm.values[:,-1:]

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

<font size=5 color='blue'>

# <center> Artificial Neural Networks </center>


<font size=4 color='blue'>

# <center> Implemented using the framework Pytorch </center>


<font size=4 color='mediumvioletred'>
   
[Pytorch](https://pytorch.org/)

<font size=5>
    CPU

`conda install pytorch torchvision torchaudio cpuonly -c pytorch`

<font size=5>
    GPU

`conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch`

<font size=5, color=brown>
    Asiri

Asiri es un módulo en Python usando Pytorch, cuya finalidad es que los estudiantes se famliaricen con la manera en que se les presentarán los notebooks en Keras, y no tengan problemas en adaptar cualquiera de los frameworks en sus proyectos   

<font size=6, color=green>
    Importación de paqueterías necesarias para trabajar con Pytorch

`sudo apt-get install graphviz`

`pip install pytorch-model-summary`

`conda install -c conda-forge scikit-learn`

`conda install -c anaconda networkx`

`conda install -c anaconda pandas`

`conda install -c lightsource2-tag collection`

`pip install torchviz`


In [ ]:
#########################Librerías de Pytorch###############################################
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.autograd import Variable

#############################################################################################

##########################Liberías complementarias###########################################
import matplotlib.pyplot as plt
from collections import OrderedDict
from collections import namedtuple
import json
import itertools
from itertools import product
from IPython import display
import pytorch_model_summary as pms
import time
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
import networkx as nx
from torchviz import make_dot
from itertools import chain
##############################################################################################

###########################Módulo creado para la clase########################################
import asiri as ai
##############################################################################################

###########################Configuraciones complementarias ###################################

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True) #por default, ya viene con True, pero para asegurar

<font size=5, color=red>
    Reproducibilidad en Pytorch

In [ ]:
seed=0
torch.manual_seed(seed)

# Conversión de Pandas a tensores de PyTorch

In [ ]:
print(type(df_train))

Vamos a usar como puente para la conversión la librería de Numpy. Procedemos a convertir los datos a arreglos de numpy

In [ ]:
df_train_np=np.array(df_train)

Así, ya podemos convertir directamente a un tensor de PyTorch

In [ ]:
df_train_pt=torch.from_numpy(df_train_np)

In [ ]:
df_train_pt.shape

Hacemos lo mismo para el test

In [ ]:
df_test_np=np.array(df_test)
df_test_pt=torch.from_numpy(df_test_np)
df_test_pt.shape

Normalizamos nuestros datos con Pytorch

In [ ]:
def normaliza(tensor):
    return (tensor-torch.mean(tensor))/torch.std(tensor)

In [ ]:
df_train_norm=normaliza(df_train_pt)
df_test_norm=normaliza(df_test_pt)

In [ ]:
df_train_norm

## Construcción del dataset

In [ ]:
train_x = df_train_norm[:,:-1]
train_y = df_train_norm[:,-1:]

In [ ]:
test_x = df_test_norm[:,:-1]
test_y = df_test_norm[:,-1:]

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [ ]:
print(test_x.shape[0])

In [ ]:
from torch.utils.data import random_split
dataset=[(i,j) for i, j in zip(train_x, train_y)]
testset=[(i,j) for i, j in zip(test_x, test_y)]
validation_portion = 0.1
int_separa=int(train_x.shape[0]*(1-validation_portion))
train_set, val_set = random_split(dataset, [int_separa, train_x.shape[0]-int_separa])
len(train_set), len(val_set)

<font size=5 color='blue'>

Generating a full-connected feedforward (FFF) artificial neural network.

In [ ]:
import networkx as nx

class Network(object):
    
    def  __init__ (self,sizes):
        self.num_layers = len(sizes)
        print("It has", self.num_layers, "layers,")
        self.sizes = sizes
        print("with the following number of nodes per layer",self.sizes)
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        
    def feedforward(self, x_of_sample):
        """Return the output of the network F(x_of_sample) """        
        for b, w in zip(self.biases, self.weights):
            x_of_sample = sigmoid(np.dot(w, x_of_sample)+b)
        return x_of_sample
    
    def graph(self,sizes):
        a=[]
        ps={}
        Q = nx.Graph()
        for i in range(len(sizes)):
            Qi=nx.Graph()    
            n=sizes[i]
            nodos=np.arange(n)
            Qi.add_nodes_from(nodos)
            l_i=Qi.nodes
            Q = nx.union(Q, Qi, rename = (None, 'Q%i-'%i))
            if len(l_i)==1:
                ps['Q%i-0'%i]=[i/(len(sizes)), 1/2]
            else:
                for j in range(len(l_i)+1):
                    ps['Q%i-%i'%(i,j)]=[i/(len(sizes)),(1/(len(l_i)*len(l_i)))+(j/(len(l_i)))]
            a.insert(i,Qi)
        for i in range(len(a)-1):
            for j in range(len(a[i])):
                for k in range(len(a[i+1])):
                    Q.add_edge('Q%i-%i' %(i,j),'Q%i-%i' %(i+1,k))
        nx.draw(Q, pos = ps)
                

In [ ]:
n_x = train_x.shape[1] 
n_h = 4
n_y = train_y.shape[1]
    
layers = [n_x, n_h, n_y]
net = Network(layers)
net.graph(layers)

<font size=5 color='blue'>
    
Architecture definition. 
    
It includes weights and biases initialization, as well as the activation functions.

In [ ]:
class architecture(nn.Module):
    def __init__(self, n_x, n_h, n_y):
        super().__init__()
        self.input_nodes=n_x    #input layer has n_x nodes
        self.hlayer1_nodes=n_h  #first hidden layer has n_h nodes
        self.output_nodes=n_y   #output layer has n_y node

        #For the first hidden layer, it is necessary to indicate its input layer, which corresponds to 
        #the input layer of the network

        self.fc1=nn.Linear(self.input_nodes, self.hlayer1_nodes)
        nn.init.uniform_(self.fc1.weight)
        nn.init.zeros_(self.fc1.bias)

        self.out=nn.Linear(self.hlayer1_nodes, self.output_nodes)
        nn.init.uniform_(self.out.weight)
        nn.init.zeros_(self.out.bias)

    def forward(self, t):
        t=torch.tanh(self.fc1(t.float()))
        t=self.out(t)
        return t

<font size=5 color="blue">

Constructing the neural network model for the Learning System

In [ ]:
# Generating a model using the architecture defined for the neural network
model = architecture(n_x, n_h, n_y).to('cpu')

<font size=5 color='blue'>
Graph and summary of the architecture

In [ ]:
ai.plot_model(model, to_file='model.png', show_shapes=True, 
       show_layer_names=True)

In [ ]:
ai.summary(architecture, n_x, n_h, n_y, 1 )

<font size=5  color='blue'>
    
Compiling the model. It includes the definition of the optimizer

In [ ]:
# We define the optimizing function and their hyperparameters: learining rate(lr) in the present case

sgd=optim.SGD(model.parameters(), lr=0.01)
compile_step=ai.model_compile(optimizer=sgd, loss='mse', metrics=['accuracy'])

In [ ]:
trainsets={
    'normal': train_set
}
valsets={
    'normal': val_set
}

params = OrderedDict(
    lr=[.01],       #[0.1, 0.01, 0.001]
    batch_size=[32], #[16,32,64]
    num_workers=[1], #[0,1,2,4,8]
    device=['cpu'], #['cuda', 'cpu']
    trainset=['normal']
)


num_epochs = 200

history, network=ai.fit(params, num_epochs, model, compile_step, trainsets, valsets)

<font size=5 color='blue'>

Plots of the cost functions versus epoch    

In [ ]:
list_loss=[results['loss'] for results in history.run_data]
list_val_loss=[results['val_loss'] for results in history.run_data]
plt.plot(list_loss, 'magenta')
plt.plot(list_val_loss, 'blue')
plt.title('Cost function')
plt.ylabel('Cost', size=16)
plt.xlabel('Epoch', size=16)
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

<font size=5, color=blue>
Evaluation of the learning. This is done using the test data

In [ ]:
params = OrderedDict(
    batch_size=[test_x.shape[0]],
    num_workers=[2], #[0,1,2,4,8]
    device=['cpu'], #['cuda', 'cpu']
)

evaluations=ai.evaluate(network, testset, params, valsets, compile_step)